# 필요한 모듈 준비

In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install mecab-python3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,192 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,463 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadconten

In [4]:
import pandas as pd
import re
from konlpy.tag import Mecab
import torch
from gensim.models import word2vec
from torchtext.data.functional import to_map_style_dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

## Data Preprocessing

In [5]:
data = pd.read_csv("ratings_train.txt", sep = "\t").dropna(subset = ['document'])
data_test = pd.read_csv("ratings_test.txt", sep = "\t").dropna(subset = ['document'])
print(len(data))
print(len(data_test))

149995
49997


In [6]:
def preprocess_text(text):
  for i in range(len(text)):
    text[i] = re.sub(r"[^가-힣a-zA-Z0-9]", " ", text[i])  #한글, 알파벳, 숫자만 남기고 제거
    text[i] = re.sub(r"\s+", " ", text[i])
  return text

text = preprocess_text([x for x in list(data['document'].values)])
text_test = preprocess_text([x for x in list(data_test['document'].values)])

In [7]:
text[:5]

['아 더빙 진짜 짜증나네요 목소리',
 '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 솔직히 재미는 없다 평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

# Tokenization

In [8]:
mecab = Mecab()
corpus = []
for item in text:
  corpus += [mecab.morphs(item)]

In [9]:
corpus[:5]

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없', '다', '평점', '조정'],
 ['사이몬페그',
  '의',
  '익살',
  '스런',
  '연기',
  '가',
  '돋보였',
  '던',
  '영화',
  '스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '했',
  '던',
  '커스틴',
  '던스트',
  '가',
  '너무나',
  '도',
  '이뻐',
  '보였',
  '다']]

In [10]:
corpus_test = []
for item in text_test:
  corpus_test += [mecab.morphs(item)]

In [11]:
corpus_test[:5]

[['굳'],
 ['GDNTOPCLASSINTHECLUB'],
 ['뭐',
  '야',
  '이',
  '평점',
  '들',
  '은',
  '나쁘',
  '진',
  '않',
  '지만',
  '10',
  '점',
  '짜리',
  '는',
  '더더욱',
  '아니',
  '잖아'],
 ['지루',
  '하',
  '지',
  '는',
  '않',
  '은데',
  '완전',
  '막장',
  '임',
  '돈',
  '주',
  '고',
  '보',
  '기',
  '에',
  '는'],
 ['3',
  'D',
  '만',
  '아니',
  '었',
  '어도',
  '별',
  '다섯',
  '개',
  '줬',
  '을',
  '텐데',
  '왜',
  '3',
  'D',
  '로',
  '나와서',
  '제',
  '심기',
  '를',
  '불편',
  '하',
  '게',
  '하',
  '죠']]

# Word2Vec

In [12]:
feature_size = 100
window_context = 30
min_word_count = 3
sample = 1e-3

w2v_model = word2vec.Word2Vec(corpus, vector_size=feature_size,
                              window=window_context, min_count=min_word_count,
                          sample=sample)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
embedding_matrix = torch.tensor(w2v_model.wv.vectors).to(device)
embedding_matrix.shape

torch.Size([22145, 100])

# Split and get data ready

In [20]:
text_train = text[:int(len(text) * 0.85)]
text_valid = text[int(len(text) * 0.85):]
data_train = data[:int(len(text) * 0.85)]
data_valid = data[int(len(text) * 0.85):]

In [21]:
train_iter =  [[x, y] for x, y in zip(text_train, data_train['label'].values.tolist())]
valid_iter =  [[x, y] for x, y in zip(text_valid, data_valid['label'].values.tolist())]
test_iter =  [[x, y] for x, y in zip(text_test, data_test['label'].values.tolist())]

In [22]:
train_dataset = to_map_style_dataset(train_iter)
valid_dataset = to_map_style_dataset(valid_iter)
test_dataset = to_map_style_dataset(test_iter)

In [24]:
voc = w2v_model.wv.key_to_index

In [25]:
def encode(text):
  encoded = [voc[token] if token in voc else 0 for token in mecab.morphs(text)]
  return encoded

In [26]:
def collate_fn(batch):
  text_list, label_list = [], []

  for (text, label) in batch:
      text_list.append(torch.tensor(encode(text), dtype = torch.long).to(device)) #형태소 별로 분리 후 각각마다 voc 속 값으로 바꾸어줌
      label_list.append(float(label))

  text_padded = pad_sequence(text_list, padding_value=0)

  return text_padded.to(device), torch.tensor(label_list).to(device)

In [27]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=True, collate_fn = collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True, collate_fn= collate_fn)

# Model(RNN)

In [28]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):

        super().__init__()

        self.embedding = nn.Embedding.from_pretrained(embedding_matrix)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):

        embedded = self.embedding(text)

        output, hidden = self.rnn(embedded)

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))

In [29]:
INPUT_DIM = embedding_matrix.shape[0]
EMBEDDING_DIM = embedding_matrix.shape[1]
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [30]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss().to(device)
model = model.to(device)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 91,905 trainable parameters


In [33]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    #print(preds[:5])
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [34]:
def train(dataloader):
    model.train()
    epoch_loss, epoch_acc = 0, 0

    for idx, (text, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label.unsqueeze(1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        epoch_acc += binary_accuracy(predicted_label,label.unsqueeze(1)).item()  #label에 추가적 unsqueeze 해줌
        epoch_loss += loss.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [35]:
def evaluate(dataloader):
    model.eval()
    epoch_loss, epoch_acc = 0, 0
    with torch.no_grad():
        for idx, (text, label) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label.unsqueeze(1))
            epoch_acc += binary_accuracy(predicted_label,label.unsqueeze(1)).item() #label에 추가적 unsqueeze 해줌
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
EPOCHS = 10

best_valid_loss = float('inf')
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    start_time = time.time()

    train_loss, train_acc = train(train_dataloader)
    valid_loss, valid_acc = evaluate(valid_dataloader)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 27s
	Train Loss: 0.693 | Train Acc: 49.71%
	 Val. Loss: 0.693 |  Val. Acc: 49.89%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.693 |  Val. Acc: 50.24%
Epoch: 03 | Epoch Time: 0m 27s
	Train Loss: 0.693 | Train Acc: 50.21%
	 Val. Loss: 0.693 |  Val. Acc: 49.78%
Epoch: 04 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.03%
	 Val. Loss: 0.693 |  Val. Acc: 49.86%
Epoch: 05 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 49.99%
	 Val. Loss: 0.693 |  Val. Acc: 49.83%
Epoch: 06 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 49.94%
	 Val. Loss: 0.693 |  Val. Acc: 49.80%
Epoch: 07 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.11%
	 Val. Loss: 0.693 |  Val. Acc: 49.79%
Epoch: 08 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.04%
	 Val. Loss: 0.693 |  Val. Acc: 50.33%
Epoch: 09 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 49.94%
	 Val. Loss: 0.693 |  Val. Acc: 49.81%
Epoch: 10 | Epoch T

In [38]:
model.load_state_dict(torch.load('rnn-model.pt'))

test_loss, test_acc = evaluate(test_dataloader)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 49.58%


In [39]:
def predict_nsmc(model, sentence, min_len=5):
  model.eval()
  tokenized = preprocess_text([sentence])  #전처리
  tensor = torch.tensor(encode(tokenized[0])).unsqueeze(1).to(device)  #voc이 나타내는 값으로 전환. 최종적으로 tensor로 바뀜
  preds =  model(tensor)
  return torch.round(torch.sigmoid(preds)).item()

# Inference

In [40]:
sentences = []
sentences.append("잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다.")
sentences.append("한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다.")
sentences.append("티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다.")
sentences.append("이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ")
sentences.append("이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다.")
sentences.append("미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다.")
sentences.append("의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요.")
sentences.append("캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다.")
sentences.append("엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요.")
sentences.append("주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽.")

for sentence in sentences:
  print(sentence)
  print(predict_nsmc(model, sentence))

잔잔한 감동... 오래 기억에 남아요... 우리 시대의 평범한 가족 모습이 솔직하게 담겨 있네요. 그리고 한 소녀의 성장기도 아름답게 표현했습니다.
1.0
한 사람의 삶을 한 화면에서 완전히 이해할 수 없다는 것을 더욱 명확하게 깨닫게 해주는 작품입니다. 밴 애플렉과 션 윌의 연기는 볼만하고, 복 받은 캐릭터들이 감동을 전해줍니다.
0.0
티비 드라마보다 못한 영화. 그 당시에는 티비 드라마가 한국 영화보다 더 흥미로웠다.
1.0
이 작품은 클래식을 위한 드라마나 청춘 성장 드라마와는 다른 주제를 다루고 있는데, 그 주제가 명확하지 않아 혼란스럽습니다. 현실을 고발하려는 의도는 알겠지만, 이 작품은 다소 심각한 드라마 중 하나입니다. 부채도사는 어디로..ㅠㅠ
0.0
이 작품을 통해 그들의 쾌락적인 삶을 엿볼 수 있고, 우리는 다른 형태의 쾌락을 느낍니다. 그들이 추잡하게 보일 수 있지만, 결국 우리도 돈을 추구하며 부유한 삶을 상상하는 것이 현실입니다. 이 영화는 감독의 메시지를 느끼게 합니다.
1.0
미식축구를 다시 생각하게 만든 감동적인 영화입니다. 개인적으로 감명 깊었습니다.
1.0
의외로 흥미로운 영화로, 지루하지 않은 사극 멜로였습니다. 캐릭터들이 매력적으로 표현되었어요.
1.0
캐릭터들 중에서 할아버지를 제외하고는 짜증을 유발하는 매력적인 캐릭터가 없다는 점이 아쉽습니다.
1.0
엔딩 임팩트가 부족합니다. 더 흥미로웠다면 좋았을 것 같아요.
1.0
주연배우들이 지루하고, 압축된 스토리가 조금 부족한 것 같습니다. 으윽.
0.0
